<a href="https://colab.research.google.com/github/tanthongtan/ptm/blob/master/lda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
num_topic = 50
dataset = 'nips'
method = 'lda'

In [8]:
#only for google colab
import sys
import os
if 'google.colab' in sys.modules:
    #lets see what gpu we were given
    !nvidia-smi
    #get repository
    !git clone https://github.com/tanthongtan/ptm.git
    %cd '/content/ptm'
    #get ref corp if doesn't exist
    if not os.path.isdir('wiki_final'):
        !unzip -q "/content/drive/My Drive/wiki_final.zip"
    if not os.path.isdir('mallet-2.0.8'):
        !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
        !unzip -q mallet-2.0.8.zip

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

Cloning into 'ptm'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 320 (delta 20), reused 23 (delta 6), pack-reused 276
Receiving objects: 100% (320/320), 257.43 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (152/152), done.
Checking out files: 100% (30/30), done.
/content/ptm


In [9]:
from dataset import load_data
from utils import print_summary, clustering_metrics_20news
import multiprocessing
import numpy as np

data_tr, data_te, vocab, vocab_size, num_tr = load_data(use_tfidf=False, normalize=False, dataset=dataset)

#convert to text for mallet
inverse_vocab = dict((v, i) for i, v in vocab.items())
with open('train.txt', 'w') as file:
    for idx, doc in enumerate(data_tr):
        print(idx, "0", end = ' ', file=file)
        for i, v in zip(doc.indices, doc.data):
            for j in range(v):
                print(inverse_vocab[i], end = ' ', file = file)
        print('', file = file)
        if idx % 1000 == 0:
          print(idx)

!mallet-2.0.8/bin/mallet import-file --input train.txt --output train.mallet --keep-sequence
!mallet-2.0.8/bin/mallet train-topics --input train.mallet --num-topics {num_topic} --alpha 50 --beta 0.1 --num-iterations 5000 --output-topic-keys topics --num-threads {multiprocessing.cpu_count()} --num-top-words 10 --output-doc-topics pi

topics = []
with open('topics') as file:
    for line in file:
        line = line.split()
        line = line[2:]
        topics.append(line)

print_summary(topics,method,dataset)

if dataset == '20news':
    pi = []
    with open('pi') as file:
        for line in file:
            line = line.split()
            line = line[2:]
            pi.append(line)
    pi = np.array(pi)
    pi = pi.astype(np.float)
    clustering_metrics_20news(pi)

os.remove('train.txt')
os.remove('train.mallet')
os.remove('topics')
os.remove('pi')

Streaming output truncated to the last 5000 lines.
17	1	sequence sequences gene protein structure expression set length genes string 
18	1	layer training deep neural networks learning network layers convolutional input 
19	1	signal noise signals frequency time filter source sources ica auditory 
20	1	model human causal figure objects category models example knowledge experiment 
21	1	network networks neural input output units hidden weights layer unit 
22	1	algorithm regret online algorithms problem bound setting submodular set bandit 
23	1	point points local figure region regions may small number example 
24	1	kernel kernels space data learning matrix gaussian regularization using based 
25	1	distance data space metric dimensional manifold nearest points dimensionality local 
26	1	function functions linear approximation basis order using space nonlinear vector 
27	1	convex optimization norm dual function regularization problem methods problems method 
28	1	gaussian prior bayesian post